In [1]:
!pip install konlpy #형태로 분리 위한 라이브러리
!pip install jamo #자모 분리 위한 라이브러리

In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#엑셀파일로부터 데이터프레임 생성, 불필요한 요소 제거

In [3]:
df = pd.read_excel('dataset.xlsx')
df = df.iloc[:,0:2]

#문장과 감정 (X, Y) 데이터를 분리

In [4]:
df_sentence =df['Sentence']
df_emotion = df['Emotion']

In [5]:
df_sentence.head(),df_emotion.head()

(0    언니 동생으로 부르는게 맞는 일인가요..??
 1                그냥 내 느낌일뿐겠지?
 2              아직너무초기라서 그런거죠?
 3               유치원버스 사고 낫다던데
 4                 근데 원래이런거맞나요
 Name: Sentence, dtype: object, 0    공포
 1    공포
 2    공포
 3    공포
 4    공포
 Name: Emotion, dtype: object)

#df_emotion을 학습을 위해 원-핫 인코딩

In [6]:
df_emotion = pd.get_dummies(df_emotion)

In [7]:
df_emotion

,공포,놀람,분노,슬픔,중립,행복,혐오
0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...
38589,0,0,0,0,0,0,1
38590,0,0,0,0,0,0,1
38591,0,0,0,0,0,0,1
38592,0,0,0,0,0,0,1


#df_sentence를 학습을 위해 토큰화 이전에 형태소 분리


In [8]:
#분리 전 df-->list형으로 포맷 변경
array_sentence = np.array(df_sentence.to_list())

In [9]:
array_sentence[0]

'언니 동생으로 부르는게 맞는 일인가요..??'

In [10]:
#형태소 분리를 위한 객체 생성
from konlpy.tag import Okt
okt = Okt()

In [11]:
array_sentence_morphs = array_sentence

In [12]:
array_sentence_morphs[0],array_sentence[0]

('언니 동생으로 부르는게 맞는 일인가요..??', '언니 동생으로 부르는게 맞는 일인가요..??')

In [13]:
#형태소 분리
for i, document in enumerate(array_sentence_morphs):
    clean_words = []
    for word in okt.morphs(document): 
        clean_words.append(word)
    print(clean_words)
    document = ' '.join(clean_words)
    print(document)
    array_sentence_morphs[i] = document

Streaming output truncated to the last 5000 lines.
['깜빡이', '안', '켜면', '범칙', '금', '좀', '세게', '물려라']
깜빡이 안 켜면 범칙 금 좀 세게 물려라
['이랜드', '옷', '불매', '해야겠네요', '.']
이랜드 옷 불매 해야겠네요 .
['뭐', '큰', '약점', '잡힌거', '있으', '신가', '어이', '가', '없네']
뭐 큰 약점 잡힌거 있으 신가 어이 가 없네
['그네', '야', '나대지말고', '쳐', '내려와']
그네 야 나대지말고 쳐 내려와
['우', '병', '우', '나오면', '??', 'ㅎㅎ']
우 병 우 나오면 ?? ㅎㅎ
['그래요', '뇬', '드디어', '걸려', '들었어', '쓰레기', '조윤선', '!!!!!']
그래요 뇬 드디어 걸려 들었어 쓰레기 조윤선 !!!!!
['내', '가', '안', '사주니까', '저래됫', '네', '.....']
내 가 안 사주니까 저래됫 네 .....
['구라', '청', '.', '역시', '나', '틀렀어']
구라 청 . 역시 나 틀렀어
['일베', '나', '메', '갈이', '나', '너무', '극단', '적', '성향', '을', '가지고있는지', '말', '이', '안통', '하더라', 'ㅋㅋ']
일베 나 메 갈이 나 너무 극단 적 성향 을 가지고있는지 말 이 안통 하더라 ㅋㅋ
['하는', '꼬', '라지', '가', '미국', '을', '몇', '년', '은', '퇴보하게', '만드는듯']
하는 꼬 라지 가 미국 을 몇 년 은 퇴보하게 만드는듯
['둘다', '욕심', '존나', '게', '많아', '보인다', 'ㅋㅋ']
둘다 욕심 존나 게 많아 보인다 ㅋㅋ
['플로리다', '도', '트럼프', '한테', '넘어가고', '선거인단', '60', '명', '이상', '차이날거', '같은데', '캘리포니아', '에서', '클린턴', '이', '이긴', '다해', '도', '쉽지', '않을', '듯']
플로리다 도

In [14]:
array_sentence_morphs[0]

'언니 동생 으로 부르는게 맞는 일 인가요 ..??'

#토큰화 및 정수화

In [15]:
array_sentence[0]

'언니 동생 으로 부르는게 맞는 일 인가요 ..??'

In [16]:
array_sentence_morphs[0]

'언니 동생 으로 부르는게 맞는 일 인가요 ..??'

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer() #토큰 객체 생성, num_word = 빈도수 상위 100개의 단어를 토큰화 한다는 뜻, default=-1(전체)
                        #이미 토큰화된 단어 이외의 새로운 단어는 보존하지 않기 위해  oov_token = 'OOV'을 주지 않음 

In [18]:
#형태소+자모 따로였던 리스트를 하나로 병합시키는 함수
'''
def merge_list(*args, fill_value = None):
  max_length = max([len(lst) for lst in args])
  merged = []
  for i in range(max_length):
    merged.append([
      args[k][i] if i < len(args[k]) else fill_value for k in range(len(args))
    ])
  return merged
  '''

'\ndef merge_list(*args, fill_value = None):\n  max_length = max([len(lst) for lst in args])\n  merged = []\n  for i in range(max_length):\n    merged.append([\n      args[k][i] if i < len(args[k]) else fill_value for k in range(len(args))\n    ])\n  return merged\n  '

In [19]:
'''
merged_array = merge_list(array_sentence_morphs,array_sentence_syllable)
#list(map(list.__add__, array_sentence_morphs, array_sentence_syllable))
'''

'\nmerged_array = merge_list(array_sentence_morphs,array_sentence_syllable)\n#list(map(list.__add__, array_sentence_morphs, array_sentence_syllable))\n'

In [20]:
tokenizer.fit_on_texts(array_sentence_morphs) #-->morphs 와 syllable을 정수화해야하는데, 하나의 리스트밖에 적용되지 않음, 위의 리스트는 오류발생. 해결 필요
                                              #토크나이징이 원하는 대로 나오지 않기 때문에, 형태소, 자모 따로 학습시킨뒤 결과값 2개를 가지고 사용해보기로 함  

In [21]:
tokenizer.word_index

{'이': 1,
 '가': 2,
 '들': 3,
 '에': 4,
 '도': 5,
 '을': 6,
 '은': 7,
 '는': 8,
 '를': 9,
 '의': 10,
 '다': 11,
 '안': 12,
 '한': 13,
 '으로': 14,
 '진짜': 15,
 '로': 16,
 '너무': 17,
 '에서': 18,
 '요': 19,
 '못': 20,
 '저': 21,
 '하고': 22,
 '사람': 23,
 '만': 24,
 '나': 25,
 '고': 26,
 '잘': 27,
 '아': 28,
 'ㅠㅠ': 29,
 '그': 30,
 '하는': 31,
 '정말': 32,
 '왜': 33,
 '것': 34,
 '말': 35,
 '네': 36,
 '더': 37,
 '때': 38,
 '제': 39,
 '게': 40,
 '거': 41,
 '내': 42,
 '생각': 43,
 '지': 44,
 'ㅋㅋ': 45,
 '이다': 46,
 '뭐': 47,
 '와': 48,
 '라': 49,
 '좀': 50,
 '일': 51,
 '인데': 52,
 '까지': 53,
 '돈': 54,
 '나라': 55,
 '할': 56,
 '하': 57,
 '국민': 58,
 '해': 59,
 '지금': 60,
 '인': 61,
 '이런': 62,
 '적': 63,
 '어떻게': 64,
 '그냥': 65,
 '니': 66,
 '개': 67,
 '많이': 68,
 '면': 69,
 '이제': 70,
 '야': 71,
 '한테': 72,
 '과': 73,
 '하면': 74,
 'ㅋㅋㅋ': 75,
 '서': 76,
 '님': 77,
 '우리': 78,
 '해서': 79,
 '부터': 80,
 '근데': 81,
 '또': 82,
 '해야': 83,
 'ㅠ': 84,
 '이나': 85,
 '놈': 86,
 '오늘': 87,
 '보다': 88,
 '여자': 89,
 '대통령': 90,
 '0': 91,
 '이네': 92,
 '랑': 93,
 '1': 94,
 '순': 95,
 '같은': 96,
 '한국': 97

In [22]:
#정수화된 토큰을 이용해 리스트를 정수화
sequenced_morphs = tokenizer.texts_to_sequences(array_sentence_morphs)

#정수화된 리스트 데이터들의 길이를 맞추기 위해 zero-padding 진행

In [23]:
padded_morphs = pad_sequences(sequenced_morphs, padding='pre', maxlen=100)
                #maxlen은 문장 내 최대길이에 맞추거나, 우리가 만들 서비스 문장의 최대허용길이로 만들자

In [24]:
padded_morphs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,  765,  972,   14, 7700, 1462,   51,
        341], dtype=int32)

In [25]:
padded_morphs.shape

(38594, 100)

#전처리가 끝난 데이터를 train,test 셋으로 분리

In [26]:
train_x, test_x, train_y, test_y =train_test_split(padded_morphs,df_emotion)

In [27]:
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((28945, 100), (9649, 100), (28945, 7), (9649, 7))

#모델 설계 (2개의 input(morphs, syllable)을 사용하는 multi-input CNN 기반으로 제작예정)
#->토크나이즈의 한계로 형태소 RNN, 자모 RNN 2개 모델 생성 후 두개의 결과값을 비교해봄
* 1개 인풋(28945,100)
* 7개 아웃풋(감정의 종류:7개)


In [28]:
from tensorflow.keras import models, layers

In [29]:
model = models.Sequential()

In [30]:
model.add(layers.SimpleRNN(32, input_shape=(28945, 100)))
model.add(layers.Dense(units=7, activation='sigmoid'))

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 32)                4256      
_________________________________________________________________
dense (Dense)                (None, 7)                 231       
Total params: 4,487
Trainable params: 4,487
Non-trainable params: 0
_________________________________________________________________


# 모델 학습

In [31]:
model.compile(optimizer='Adam', loss='mse', metrics=['accuracy'])

#모델 테스트

In [32]:
model.fit(train_x, train_y, batch_size=10, epochs=10, validation_split=0.2)

Epoch 1/10


ValueError: ignored